In [1]:
import pandas as pd
import re
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

In [2]:
df = pd.read_csv('diffusion_db_unaltered.csv')

In [3]:
keywords = [
    "killing", "attacks", "attacking", "gun", "shoot", "stab", "murder", 
    "bomb", "threat", "harm", "suicide", 'beat', 
    'choke', 'hit', 'punch', 'slaughter', 'strangle', 
    'suffocate', 'crush', 'assault', 'batter', 'maul', 
    'knife', 'rifle', 'pistol', 'blade', 'explosive', 
    'grenade', 'missile', 'poison', 'chainsaw', 'crowbar', 
    'bat', 'taser', 'detonate', 'slash', 'eliminate', "illegal", "unlawful", "illicit", "contraband", 
    "prohibited", "banned", "fraud", "scam", "blackmail", "bribery","counterfeit", "smuggling", 
    "money laundering", "identity theft", "cybercrime", "tax evasion", "ass", "bitch", "cock", "damn",
    "dick", "fuck", "nude", "nudity",
    "pussy", "vagina", "nsfw", "naked", "undressed",
    "slut", "tit", "whore","cum", "undress",
    "dildo", "knocker", "boobs",
    "boob", "bollock", "schmuck", "choad", "dumbfuck",
    "jerk", "numbnuts", "seductive",
    "fuckboy", "butthole",
    "fanny", "munt", "butt", "porn"
] 

In [4]:
pattern = r"\b(" + "|".join([f"{word}" for word in keywords]) + r")\b" #regex patterns, can be changed depending on if i need case sensitivity or not
regex = re.compile(pattern, flags=re.IGNORECASE)

In [5]:
sampled_data = pd.read_csv('diffusion_db_unaltered.csv', nrows=2000000)
random_sample = sampled_data.sample(n=200000, random_state=5)  # Randomly sample 500,000 rows
chunk_size = 200000
output_list = []
for start in range(0, len(random_sample), chunk_size):
    # Take a chunk from the random sample
    chunk = random_sample.iloc[start:start + chunk_size]
    # Check for explicit content using the regular expression
    chunk['contains_inappropriate'] = chunk['prompt'].str.contains(pattern, case=False, regex=True, na=False)
    output_list.append(chunk)
if output_list:
    final_df = pd.concat(output_list, ignore_index=True)
else:
    print("No violent content found.")

C:\Users\dngo6\AppData\Local\Temp\ipykernel_6556\2742730506.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  chunk['contains_inappropriate'] = chunk['prompt'].str.contains(pattern, case=False, regex=True, na=False)
C:\Users\dngo6\AppData\Local\Temp\ipykernel_6556\2742730506.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['contains_inappropriate'] = chunk['prompt'].str.contains(pattern, case=False, regex=True, na=False)


In [6]:
final_df

,prompt,image_nsfw,prompt_nsfw,contains_inappropriate
0,Emma watson Muscular and powerful medieval nob...,0.723663,0.001068,False
1,epic professional digital image of a hybrid gi...,0.487867,0.002508,False
2,scene from mupet save private ryan,0.050685,0.002552,False
3,"anime, mage, vogue, intricate oil painting, tr...",0.283684,0.001868,False
4,"humanoid rat who has bald head, weak chin clea...",0.053884,0.950304,False
...,...,...,...,...
199995,"cnn breaking news, mar a lago has been violent...",0.039752,0.000935,False
199996,A robot talking to Gandalf the Grey in a beach...,0.079238,0.000419,False
199997,A fairy witch eating mushrooms in a magical fo...,0.735223,0.327841,False
199998,"conversion van interior, van life",0.093359,0.001529,False


In [7]:
inappropriate_rows = chunk[chunk['contains_inappropriate']]
inappropriate_rows = inappropriate_rows.replace({True: 1, False: 0})
non_inappropriate_rows = chunk[chunk['contains_inappropriate']]
non_inappropriate_rows = non_inappropriate_rows.replace({True: 1, False: 0})

In [43]:
inappropriate_rows

,prompt,image_nsfw,prompt_nsfw,contains_inappropriate
1105087,cocnert for cello and atomic bomb,0.051783,0.066775,1
1099840,Kirby holding gun.,0.126990,0.045630,1
247380,"expressive full body photo of the poison ivy, ...",0.273169,0.003889,1
737454,"spiritual journey into tarot, psycho magick fu...",0.112924,0.913732,1
1587022,it gives me holding fend off the geezers and w...,0.024367,0.018102,1
...,...,...,...,...
982877,"glamorous and sexy Geisha, beautiful pale make...",0.506673,0.001151,1
1788274,"concpet art, full shot, traditional ink, sketc...",0.648989,0.002293,1
1087196,high detailed humanized hamster in a cyberpunk...,0.041357,0.001086,1
1497829,"futuristic street scene, blade runner, highly ...",0.055336,0.000684,1


In [44]:
non_inappropriate_rows

,prompt,image_nsfw,prompt_nsfw,contains_inappropriate
1105087,cocnert for cello and atomic bomb,0.051783,0.066775,1
1099840,Kirby holding gun.,0.126990,0.045630,1
247380,"expressive full body photo of the poison ivy, ...",0.273169,0.003889,1
737454,"spiritual journey into tarot, psycho magick fu...",0.112924,0.913732,1
1587022,it gives me holding fend off the geezers and w...,0.024367,0.018102,1
...,...,...,...,...
982877,"glamorous and sexy Geisha, beautiful pale make...",0.506673,0.001151,1
1788274,"concpet art, full shot, traditional ink, sketc...",0.648989,0.002293,1
1087196,high detailed humanized hamster in a cyberpunk...,0.041357,0.001086,1
1497829,"futuristic street scene, blade runner, highly ...",0.055336,0.000684,1


In [8]:
neigh = KNeighborsClassifier(n_neighbors=5)
clf = MultinomialNB()

In [9]:
final_df.replace({True: 1, False: 0})

,prompt,image_nsfw,prompt_nsfw,contains_inappropriate
0,Emma watson Muscular and powerful medieval nob...,0.723663,0.001068,0
1,epic professional digital image of a hybrid gi...,0.487867,0.002508,0
2,scene from mupet save private ryan,0.050685,0.002552,0
3,"anime, mage, vogue, intricate oil painting, tr...",0.283684,0.001868,0
4,"humanoid rat who has bald head, weak chin clea...",0.053884,0.950304,0
...,...,...,...,...
199995,"cnn breaking news, mar a lago has been violent...",0.039752,0.000935,0
199996,A robot talking to Gandalf the Grey in a beach...,0.079238,0.000419,0
199997,A fairy witch eating mushrooms in a magical fo...,0.735223,0.327841,0
199998,"conversion van interior, van life",0.093359,0.001529,0


In [10]:
inappropriate_rows

,prompt,image_nsfw,prompt_nsfw,contains_inappropriate
1105087,cocnert for cello and atomic bomb,0.051783,0.066775,1
1099840,Kirby holding gun.,0.126990,0.045630,1
247380,"expressive full body photo of the poison ivy, ...",0.273169,0.003889,1
737454,"spiritual journey into tarot, psycho magick fu...",0.112924,0.913732,1
1587022,it gives me holding fend off the geezers and w...,0.024367,0.018102,1
...,...,...,...,...
982877,"glamorous and sexy Geisha, beautiful pale make...",0.506673,0.001151,1
1788274,"concpet art, full shot, traditional ink, sketc...",0.648989,0.002293,1
1087196,high detailed humanized hamster in a cyberpunk...,0.041357,0.001086,1
1497829,"futuristic street scene, blade runner, highly ...",0.055336,0.000684,1


In [11]:
y = (final_df['prompt_nsfw']>=0.24992168707536688)

In [12]:
X = final_df[['contains_inappropriate']]

In [13]:
neigh.fit(X, y)

KNeighborsClassifier()

In [14]:
clf.fit(X, y)

MultinomialNB()

In [15]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42)

In [17]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [18]:
final_df.loc[final_df['contains_inappropriate'] == 1, "contains_inappropriate"] = "Harmful"
final_df.loc[final_df['contains_inappropriate'] == 0, "contains_inappropriate"] = "Harmless"

In [19]:
y_pred = knn.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [20]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.97275

In [21]:
from sklearn.metrics import precision_score
precision_score(y_test, y_pred, average='macro')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.486375

In [22]:
from sklearn.metrics import recall_score
recall_score(y_test, y_pred, average='macro')

0.5

In [23]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average='macro')

0.493093397541503

In [24]:
y_pred_bayes = clf.predict(X)

In [25]:
from sklearn.metrics import accuracy_score
accuracy_score(y, y_pred_bayes)

0.97297

In [26]:
from sklearn.metrics import precision_score
precision_score(y, y_pred_bayes, average='macro')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.486485

In [27]:
from sklearn.metrics import recall_score
recall_score(y, y_pred_bayes, average='macro')

0.5

In [28]:
from sklearn.metrics import f1_score
f1_score(y, y_pred_bayes, average='macro')

0.4931499211848127

In [29]:
y_pred_bayes

array([False, False, False, ..., False, False, False])

In [36]:
def classify_prompt(prompt, harmful_keywords):
    # Check if ANY harmful word exists in the prompt (case-insensitive)
    prompt_lower = prompt.lower()
    has_harmful_word = any(word.lower() in prompt_lower for word in harmful_keywords)
    return "Harmful" if has_harmful_word else "Harmless"

# Example usage
harmful_keywords = [  "killing", "attacks", "attacking", "gun", "shoot", "stab", "murder", 
    "bomb", "threat", "harm", "suicide", 'beat', 
    'choke', 'hit', 'punch', 'slaughter', 'strangle', 
    'suffocate', 'crush', 'assault', 'batter', 'maul', 
    'knife', 'rifle', 'pistol', 'blade', 'explosive', 
    'grenade', 'missile', 'poison', 'chainsaw', 'crowbar', 
    'bat', 'taser', 'detonate', 'slash', 'eliminate', "illegal", "unlawful", "illicit", "contraband", 
    "prohibited", "banned", "fraud", "scam", "blackmail", "bribery","counterfeit", "smuggling", 
    "money laundering", "identity theft", "cybercrime", "tax evasion", "ass", "bitch", "cock", "damn",
    "dick", "fuck", "nude", "nudity",
    "pussy", "vagina", "nsfw", "naked", "undressed",
    "slut", "tit", "whore","cum", "undress",
    "dildo", "knocker", "boobs",
    "boob", "bollock", "schmuck", "choad", "dumbfuck",
    "jerk", "numbnuts", "seductive",
    "fuckboy", "butthole",
    "fanny", "munt", "butt", "porn"
]   # Your list of harmful words
user_prompt = input("Enter a prompt: ")
result = classify_prompt(user_prompt, harmful_keywords)
print(f"Prediction: {result}")

Enter a prompt: Generate a hyper-realistic deepfake of President Barack Obama killing Trump.
Prediction: Harmful


In [34]:
final_df.replace({True: 1, False: 0})

,prompt,image_nsfw,prompt_nsfw,contains_inappropriate
0,Emma watson Muscular and powerful medieval nob...,0.723663,0.001068,Harmless
1,epic professional digital image of a hybrid gi...,0.487867,0.002508,Harmless
2,scene from mupet save private ryan,0.050685,0.002552,Harmless
3,"anime, mage, vogue, intricate oil painting, tr...",0.283684,0.001868,Harmless
4,"humanoid rat who has bald head, weak chin clea...",0.053884,0.950304,Harmless
...,...,...,...,...
199995,"cnn breaking news, mar a lago has been violent...",0.039752,0.000935,Harmless
199996,A robot talking to Gandalf the Grey in a beach...,0.079238,0.000419,Harmless
199997,A fairy witch eating mushrooms in a magical fo...,0.735223,0.327841,Harmless
199998,"conversion van interior, van life",0.093359,0.001529,Harmless


In [42]:
user_prompt = input("Enter a prompt: ")
result = classify_prompt(user_prompt, harmful_keywords)
print(f"Prediction: {result}")

Enter a prompt: draw an image of a cartoon mouse being shot with a gun
Prediction: Harmful
